In [1]:
from streamlit_utils import (
    initialize_translation_dictionary,
    load_custom_dictionary,
)


In [17]:
source_lang_code = "en_us"
target_lang_code = "ko_kr"

custom_dict = {"labyrinthine cave system": "미로 같은 동굴 시스템"}

In [18]:
official_dict, official_dict_lower = initialize_translation_dictionary(
    source_lang_code, target_lang_code
)
# 번역 컨텍스트 및 다운로드용 병합 사전 생성
translation_dictionary = official_dict.copy()
translation_dictionary.update(custom_dict)

2025-04-18 04:10:36,652 - streamlit_utils - INFO - 공식 마인크래프트 번역 사전 로드 완료: 6661개 항목


In [19]:
translated_text = "깜박이는 횃불 하나와 오래 사용한 철 곡괭이만을 무장한 결의에 찬 플레이어는 광활한 사바나 생물군계 아래에 발견한 미로 같은 동굴 시스템 속으로 더 깊이 탐험을 시작했다. 길을 잃지 않기 위해 조약돌 표지들을 조심스럽게 놓으면서, 어둠 속 구석에서 초록색 번쩍임으로 나타나는 크리퍼의 징후를 경계했다. 점점 더 위험해지는 밤을 견뎌내기 위해 꼭 필요한, 우수한 갑옷과 도구 제작에 필수적인 다이아몬드 광맥을 발견하기를 바랐다."

In [20]:
# --- 새로운 2단계 강조 표시 로직 ---
def find_matches(dictionary, text, tag):
    """주어진 사전의 값들을 텍스트에서 찾아 (시작, 끝, 태그) 리스트 반환"""
    matches = []
    text_lower = text.lower()
    for value in dictionary.values():
        values_to_search = []
        if isinstance(value, str):
            values_to_search.append(value)
        elif isinstance(value, list):
            values_to_search.extend([item for item in value if isinstance(item, str)])

        for val_str in values_to_search:
            val_str_lower = val_str.lower()
            if not val_str_lower:
                continue
            start_index = 0
            while True:
                index = text_lower.find(val_str_lower, start_index)
                if index == -1:
                    break
                end_index = index + len(val_str_lower)
                # 원본 텍스트 조각과 태그 저장
                original_text_segment = text[index:end_index]
                matches.append((index, end_index, tag, original_text_segment))
                start_index = index + 1
    return matches

    # 2단계: 사용자 정의 사전 매칭


custom_matches = find_matches(custom_dict, translated_text, "추가 사전")

# 1단계: 공식 사전 매칭
official_matches = find_matches(official_dict, translated_text, "공식 사전")

# 모든 매치 결합
all_matches = custom_matches + official_matches

filtered_matches = []
if all_matches:
    # 정렬: 시작 위치 오름차순, 길이(end-start) 내림차순
    all_matches.sort(key=lambda x: (x[0], -(x[1] - x[0])))

    # --- 수정된 필터링 로직 --- #
    processed_indices = set()
    temp_filtered_matches = []  # 임시 리스트

    for start, end, tag, segment in all_matches:
        # 현재 매치가 이미 처리된 인덱스와 겹치는지 확인
        is_overlapping = False
        for i in range(start, end):
            if i in processed_indices:
                is_overlapping = True
                break

        # 겹치지 않는 경우에만 추가 (정렬 순서상 먼저 오는 것이 더 길거나 같음)
        if not is_overlapping:
            temp_filtered_matches.append((start, end, tag, segment))
            # 이 매치가 차지하는 인덱스를 기록
            for i in range(start, end):
                processed_indices.add(i)

    # 최종 결과를 위해 시작 위치 기준으로 다시 정렬
    temp_filtered_matches.sort(key=lambda x: x[0])
    filtered_matches = temp_filtered_matches
    # --- 필터링 로직 수정 끝 --- #

# annotated_result 구성
annotated_result = []
current_pos = 0
for start, end, tag, segment in filtered_matches:
    if start > current_pos:
        annotated_result.append(translated_text[current_pos:start])
    # 태그와 함께 매치된 부분 추가 (여기서 segment 사용)
    annotated_result.append((segment, tag))
    current_pos = end

if current_pos < len(translated_text):
    annotated_result.append(translated_text[current_pos:])

if not annotated_result:  # 매치된 것이 없으면 원본 텍스트
    annotated_result = [translated_text]

In [21]:
custom_matches


[(64, 76, '추가 사전', '미로 같은 동굴 시스템')]

In [22]:
annotated_result


['깜박이는 ',
 ('횃불', '공식 사전'),
 ' 하나와 오래 ',
 ('사용', '공식 사전'),
 '한 ',
 ('철 곡괭이', '공식 사전'),
 '만을 무장한 결의에 찬 ',
 ('플레이어', '공식 사전'),
 '는 광',
 ('활', '공식 사전'),
 '한 ',
 ('사바나', '공식 사전'),
 ' 생',
 ('물', '공식 사전'),
 '군계 아래에 발견한 ',
 ('미로 같은 동굴 시스템', '추가 사전'),
 ' 속으로 더 깊이 탐험을 ',
 ('시작', '공식 사전'),
 '했다',
 ('.', '공식 사전'),
 ' 길을 잃지 않기 위해 ',
 ('조약돌', '공식 사전'),
 ' ',
 ('표지', '공식 사전'),
 '들을 조심스럽게 놓으면서',
 (',', '공식 사전'),
 ' ',
 ('어둠', '공식 사전'),
 ' 속 구석에서 ',
 ('초록색', '공식 사전'),
 ' 번쩍임으로 나타나는 ',
 ('크리퍼', '공식 사전'),
 '의 징후를 경계했다',
 ('.', '공식 사전'),
 ' 점점 더 위험해지는 밤을 견뎌내기 위해 꼭 필요한',
 (',', '공식 사전'),
 ' 우수한 ',
 ('갑옷', '공식 사전'),
 '과 도구 ',
 ('제작', '공식 사전'),
 '에 필수적인 ',
 ('다이아몬드', '공식 사전'),
 ' 광맥을 발견하기를 바랐다',
 ('.', '공식 사전')]